# How to call functions with chat models

This notebook covers how to use the Chat Completions API in combination with external functions to extend the capabilities of GPT models.

`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to the provided specifications. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

If the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `function_call` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `function_call` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_message"` in the response, as well as a `function_call` object that has the name of the function and the generated function arguments.

### Overview

- **How to generate function arguments:** Specify a set of functions and make calls to the API with specific instructions to generate function arguments.
- **How to call functions with model generated arguments:** Close the loop by actually executing specified functions.

## How to generate function arguments

In [ ]:
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor 
!pip install openai
!pip install requests

In [2]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"

## Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [3]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )


## Basic concepts

Let's create some function specifications to interface with a hypothetical weather API. Later we'll pass these function specification to the Chat Completions API in order to generate function arguments that adhere to the specification.

In [5]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
    {
        "name": "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required": ["location", "format", "num_days"]
        },
    },
]

If we prompt the model about the current weather, it will respond with some clarifying questions.

In [6]:
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "What's the weather like today")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
conversation.add_message(assistant_message["role"], assistant_message["content"])
assistant_message

{'role': 'assistant',
 'content': 'I can help you with that. Could you please provide me with your location?'}

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [7]:
conversation.add_message("user", "I'm in Glasgow, Scotland.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
conversation.add_message(assistant_message["role"], assistant_message["content"])
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}'}}

By prompting it differently, we can get it to target the other function we've told it about.

In [8]:
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "what is the weather going to be like in Glasgow, Scotland over the next x days")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
conversation.add_message(assistant_message["role"], assistant_message["content"])
assistant_message

{'role': 'assistant',
 'content': 'Sure, I can help you with that. Please provide the number of days you would like to get the weather forecast for.'}

Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [9]:
conversation.add_message("user", "5 days")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
chat_response.json()["choices"][0]

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_n_day_weather_forecast',
   'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}'}},
 'finish_reason': 'function_call'}

We can force the model to use a specific function, for example get_n_day_weather_forecast. By doing so, we force the model to make assumptions about how to use it.

In [10]:
# in this cell we force the model to use get_n_day_weather_forecast
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "Give me a weather report for Toronto, Canada.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)
chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_n_day_weather_forecast',
  'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}'}}

In [11]:
# if we don't force the model to use get_n_day_weather_forecast it may not
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "Give me a weather report for Toronto, Canada.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions, 
)
chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n"location": "Toronto, Canada",\n"format": "celsius"\n}'}}

We can also force the model to not use a function at all. By doing so we prevent it from producing a proper function call.

In [12]:
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "Give me the current weather (use Celcius) for Toronto, Canada.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions, function_call="none"
)
chat_response.json()["choices"][0]["message"]

{'role': 'assistant',
 'content': 'Sure! I will retrieve the current weather in Toronto, Canada for you.'}

## How to call functions with model generated arguments

In our next example, we'll demonstrate how to execute functions whose inputs are model-generated, and use this to implement an agent that can answer questions for us about a database. For simplicity we'll use the [Chinook sample database](https://www.sqlitetutorial.net/sqlite-sample-database/).

*Note:* SQL generation use cases are high-risk in a production environment - models can be unreliable when generating consistent SQL syntax. A more reliable way to solve this problem may be to build a query generation API that takes the desired columns as input from the model.

### Pull SQL Database Info

First let's define some helpful utility functions to extract data from a SQLite database.

In [13]:
import sqlite3

conn = sqlite3.connect("data/Chinook.db")
print("Opened database successfully")

Opened database successfully


In [14]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts


Now can use these utility functions to extract a representation of the database schema.

In [15]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

As before, we'll define a function specification for the function we'd like the API to generate arguments for. Notice that we are inserting the database schema into the function specification. This will be important for the model to know about.

In [16]:
functions = [
    {
        "name": "ask_database",
        "description": "Use this function to answer user questions about music. Output should be a fully formed SQL query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": f"""
                            SQL query extracting info to answer the user's question.
                            SQL should be written using this database schema:
                            {database_schema_string}
                            The query should be returned in plain text, not in JSON.
                            """,
                }
            },
            "required": ["query"],
        },
    }
]

### SQL execution

Now let's implement the function that the agent will use to query the database. We also need to implement utilities to integrate the calls to the Chat Completions API with the function it is calling.

In [17]:
def chat_completion_with_function_execution(messages, functions=None):
    """This function makes a ChatCompletion API call and if a function call is requested, executes the function"""
    try:
        response = chat_completion_request(messages, functions)
        full_message = response.json()["choices"][0]
        if full_message["finish_reason"] == "function_call":
            print(f"Function generation requested, calling function")
            return call_function(messages, full_message)
        else:
            print(f"Function not required, responding to user")
            return response.json()
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return response


def call_function(messages, full_message):
    """Executes function calls using model generated function arguments."""

    # We'll add our one function here - this can be extended with any additional functions
    if full_message["message"]["function_call"]["name"] == "ask_database":
        query = eval(full_message["message"]["function_call"]["arguments"])
        print(f"Generated SQL query is: {query}")
        try:
            results = ask_database(conn, query["query"])
        except Exception as e:
            print(e)

            # This following block tries to fix any issues in query generation with a subsequent call
            messages.append(
                {
                    "role": "system",
                    "content": f"""Query: {query['query']}
The previous query received the error {e}. 
Please return a fixed SQL query in plain text.
Your response should consist of ONLY the SQL query with the separator sql_start at the beginning and sql_end at the end""",
                }
            )
            response = chat_completion_request(messages, model="gpt-4-next")

            # Retrying with the fixed SQL query. If it fails a second time we exit.
            try:
                cleaned_query = response.json()["choices"][0]["message"][
                    "content"
                ].split("sql_start")[1]
                cleaned_query = cleaned_query.split("sql_end")[0]
                print(cleaned_query)
                results = ask_database(conn, cleaned_query)
                print(results)
                print("Got on second try")

            except Exception as e:
                print("Second failure, exiting")

                print(f"Function execution failed")
                print(f"Error message: {e}")

        messages.append(
            {"role": "function", "name": "ask_database", "content": str(results)}
        )

        try:
            response = chat_completion_request(messages)
            return response.json()
        except Exception as e:
            print(type(e))
            print(e)
            raise Exception("Function chat request failed")
    else:
        raise Exception("Function does not exist and cannot be called")


def ask_database(conn, query):
    """Function to query SQLite database with provided SQL query."""
    try:
        results = conn.execute(query).fetchall()
        return results
    except Exception as e:
        raise Exception(f"SQL error: {e}")


In [18]:
agent_system_message = """Answer user questions by generating SQL queries against the Chinook Music Database."""

sql_conversation = Conversation()
sql_conversation.add_message("system", agent_system_message)
sql_conversation.add_message(
    "user", "Hi, who are the top 5 artists by number of tracks?"
)

In [19]:
chat_response = chat_completion_with_function_execution(
    sql_conversation.conversation_history, functions=functions
)
try:
    assistant_message = chat_response["choices"][0]["message"]["content"]
    print(assistant_message)
except Exception as e:
    print(e)
    print(chat_response)

Function generation requested, calling function
Prepped query is {'query': 'SELECT Artist.Name, COUNT(*) as TrackCount FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId JOIN Track ON Album.AlbumId = Track.AlbumId GROUP BY Artist.Name ORDER BY TrackCount DESC LIMIT 5'}
The top 5 artists with the highest number of tracks in the Chinook Music Database are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks

Please note that these rankings are based on the number of tracks available in the database and may not necessarily reflect an artist's overall discography.


In [20]:
sql_conversation.add_message("assistant", assistant_message)
sql_conversation.display_conversation(detailed=True)


system: You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!


user: Hi, who are the top 5 artists by number of tracks


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: The top 5 artists with the highest number of tracks in the Chinook Music Database are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks

Please note that these rankings are based on the number of tracks available in the database and may not necessarily reflect an artist's overall discography.




In [21]:
sql_conversation.add_message(
    "user", "What is the name of the album with the most tracks"
)


In [22]:
chat_response = chat_completion_with_function_execution(
    sql_conversation.conversation_history, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]["content"]
assistant_message


Function generation requested, calling function
Prepped query is {'query': 'SELECT AlbumId, Title, COUNT(TrackId) AS TrackCount FROM Album GROUP BY AlbumId ORDER BY TrackCount DESC LIMIT 1'}
SQL error: no such column: TrackId

SELECT Album.Title, COUNT(Track.TrackId) AS TrackCount 
FROM Album 
JOIN Track ON Album.AlbumId = Track.AlbumId 
GROUP BY Album.AlbumId 
ORDER BY TrackCount DESC 
LIMIT 1

[('Greatest Hits', 57)]
Got on second try


'sql_start\nSELECT Title, COUNT(TrackId) AS TrackCount \nFROM Album \nGROUP BY Title \nORDER BY TrackCount DESC \nLIMIT 1\nsql_end'

In [23]:
sql_conversation.add_message("assistant", assistant_message)


In [24]:
sql_conversation.display_conversation(detailed=True)


system: You are ChinookGPT, a helpful assistant who gets answers to user questions from the Chinook Music Database.
Provide as many details as possible to your users
Begin!


user: Hi, who are the top 5 artists by number of tracks


function: [('Iron Maiden', 213), ('U2', 135), ('Led Zeppelin', 114), ('Metallica', 112), ('Lost', 92)]


assistant: The top 5 artists with the highest number of tracks in the Chinook Music Database are:

1. Iron Maiden - 213 tracks
2. U2 - 135 tracks
3. Led Zeppelin - 114 tracks
4. Metallica - 112 tracks
5. Lost - 92 tracks

Please note that these rankings are based on the number of tracks available in the database and may not necessarily reflect an artist's overall discography.


user: What is the name of the album with the most tracks


system: Query: SELECT AlbumId, Title, COUNT(TrackId) AS TrackCount FROM Album GROUP BY AlbumId ORDER BY TrackCount DESC LIMIT 1
The previous query received the error SQL error: no such column: TrackId. 
Please return a fix